# Mule Account Detection

### Connect to Graph Server

In [211]:
import json
import os
import platform
import sys
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import pypgx as pgx

base_url = "http://graph-server:7007"
username = "graph_dev"
password = "Welcome1"

def generateToken():
    body = json.dumps({ 'username': username, 'password': password }).encode('utf8')
    headers = { 'content-type': 'application/json' }
    request = Request(base_url + '/auth/token', data=body, headers=headers)
    try:
        response = urlopen(request).read().decode('utf-8')
        return json.loads(response).get('access_token')
    except HTTPError as err:
        if err.code == 400:
            print('Authentication failed no username/password given')
        elif err.code == 401:
            print('Authentication failed invalid username/password')
        else:
            print("Server returned HTTP response code: {} for URL: {}".format(err.code, err.url))
        os._exit(1)
 
session = pgx.get_session(base_url=base_url, token=generateToken())
analyst = session.create_analyst()
print(session)

PgxSession(id: 46f6dd9f-efae-4e9c-9898-bc98d6d601b5, name: python_pgx_client)


### Get Pre-loaded Graph

In [212]:
graph = session.get_graph("Mule Account")
graph

PgxGraph(name: Mule Account, v: 14, e: 18, directed: True, memory(Mb): 0)

### Find owners of accounts

In [213]:
graph.query_pgql("""
  SELECT
    ID(a)         AS id
  , a.account_no
  , c.name
  , a.is_mule
  FROM MATCH (a)<-[:owns]-(c)
  WHERE a.type = 'Account'
""").print()

+-------------------------------------------+
| id  | account_no | name         | is_mule |
+-------------------------------------------+
| 101 | a01        | Alice Nguyen | true    |
| 102 | a02        | Bob Tanaka   |         |
| 103 | a03        | Carol Lee    |         |
+-------------------------------------------+



In [214]:
graph.destroy_vertex_property_if_exists("owner")
graph.create_vertex_property("string", "owner")

VertexProperty(name: owner, type: string, graph: Mule Account)

In [215]:
graph = graph.clone_and_execute_pgql("""
  UPDATE a SET (a.owner = c.name)
  FROM MATCH (a)<-[:owns]-(c)
  WHERE a.type = 'Account'
""")
graph

PgxGraph(name: sub-graph_231, v: 14, e: 18, directed: True, memory(Mb): 0)

In [216]:
graph.query_pgql("""
  SELECT
    ID(a)         AS id
  , a.account_no
  , a.owner
  FROM MATCH (a)
  WHERE a.type = 'Account'
""").print()

+---------------------------------+
| id  | account_no | owner        |
+---------------------------------+
| 101 | a01        | Alice Nguyen |
| 102 | a02        | Bob Tanaka   |
| 103 | a03        | Carol Lee    |
| 104 | a04        |              |
| 105 | a05        |              |
+---------------------------------+



### If the owner of this account is sharing personal information with others

In [217]:
graph.query_pgql("""
  SELECT ID(a1), COUNT(*)
  FROM MATCH (a1)<-[:owns]-(c1)-(s)-(c2)-[:owns]->(a2)
  WHERE a1 != a2 AND c1 != c2
  GROUP BY a1
""").print()

+-------------------+
| ID(a1) | COUNT(*) |
+-------------------+
| 102    | 2        |
| 103    | 1        |
| 101    | 1        |
+-------------------+



In [218]:
graph.destroy_vertex_property_if_exists("share_info")
graph.create_vertex_property("integer", "share_info")

VertexProperty(name: share_info, type: integer, graph: sub-graph_231)

In [219]:
graph2 = graph.clone_and_execute_pgql("""
  UPDATE a1 SET (a1.share_info = COUNT(*))
  FROM MATCH (a1)<-[:owns]-(c1)-(s)-(c2)-[:owns]->(a2)
  WHERE a1 != a2 AND c1 != c2
  GROUP BY a1
""")
graph2

PgxGraph(name: sub-graph_234, v: 14, e: 18, directed: True, memory(Mb): 0)

In [220]:
graph2.query_pgql("""
  SELECT
    ID(a)         AS id
  , a.account_no
  , a.owner
  , a.share_info
  , a.is_mule
  FROM MATCH (a)
  WHERE a.type = 'Account'
""").print()

+--------------------------------------------------------+
| id  | account_no | owner        | share_info | is_mule |
+--------------------------------------------------------+
| 101 | a01        | Alice Nguyen | 1          | true    |
| 102 | a02        | Bob Tanaka   | 2          |         |
| 103 | a03        | Carol Lee    | 1          |         |
| 104 | a04        |              | 0          |         |
| 105 | a05        |              | 0          |         |
+--------------------------------------------------------+



### How many fraud accounts exist in the same money transfer community

In [221]:
graph3 = graph2.filter(pgx.EdgeFilter("edge.label()='transfer'"))
graph3

PgxGraph(name: sub-graph_236, v: 5, e: 6, directed: True, memory(Mb): 0)

In [222]:
result = analyst.scc_kosaraju(graph3)
result

PgxPartition(graph: sub-graph_236, components: 4)

In [223]:
graph3.query_pgql("""
  SELECT
    ID(a)          AS id
  , a.account_no
  , a.owner
  , a.share_info
  , a.scc_kosaraju AS component_id
  , a.is_mule
  FROM MATCH (a)
  WHERE a.type = 'Account'
""").print()

+-----------------------------------------------------------------------+
| id  | account_no | owner        | share_info | component_id | is_mule |
+-----------------------------------------------------------------------+
| 101 | a01        | Alice Nguyen | 1          | 3            | true    |
| 102 | a02        | Bob Tanaka   | 2          | 3            |         |
| 103 | a03        | Carol Lee    | 1          | 2            |         |
| 104 | a04        |              | 0          | 1            |         |
| 105 | a05        |              | 0          | 0            |         |
+-----------------------------------------------------------------------+



### Closeness to known fraud accounts in money transfer

In [224]:
rs = graph3.query_pgql("SELECT ID(a) FROM MATCH (a) WHERE a.is_mule = 'true'")
vertex = graph3.get_vertex(rs.get_row(0))
graph3.destroy_vertex_property_if_exists("ppr")
analyst.personalized_pagerank(graph3, vertex, rank="ppr")

VertexProperty(name: ppr, type: double, graph: sub-graph_236)

In [225]:
graph3.query_pgql("""
  SELECT
    ID(a)          AS id
  , a.account_no
  , a.owner
  , a.share_info
  , a.scc_kosaraju AS component_id
  , a.ppr          AS mule_closeness
  , a.is_mule
  FROM MATCH (a)
  WHERE a.type = 'Account'
""").print()

+-------------------------------------------------------------------------------------------+
| id  | account_no | owner        | share_info | component_id | mule_closeness    | is_mule |
+-------------------------------------------------------------------------------------------+
| 101 | a01        | Alice Nguyen | 1          | 3            | 0.540319263320892 | true    |
| 102 | a02        | Bob Tanaka   | 2          | 3            | 0.459680736679108 |         |
| 103 | a03        | Carol Lee    | 1          | 2            | 0.0               |         |
| 104 | a04        |              | 0          | 1            | 0.0               |         |
| 105 | a05        |              | 0          | 0            | 0.0               |         |
+-------------------------------------------------------------------------------------------+



### Store Query Result to Data Frame

In [226]:
rs = graph3.query_pgql("""
  SELECT
    ID(a)          AS id
  , a.account_no
  , a.owner
  , a.share_info
  , a.scc_kosaraju AS component_id
  , a.ppr          AS mule_closeness
  , a.is_mule
  FROM MATCH (a)
  WHERE a.type = 'Account'
""")

In [227]:
import numpy as np
import pandas as pd
def pgql_to_pandas(pgql):
    num_rows = pgql.num_results
    print("Converted to PGQL result set")
    # Fetch all vectors into the local RAM session
    temp_result = list(pgql._result_set_util.toList(pgql._pgql_result_set,0,num_rows))
    print("Fetched all rows")
    # Convert the result into numpy arrays
    temp_result2 = np.array(list(map(list,temp_result)))
    print("Converted all rows to lists")
    # Create a pandas dataframe containing the vectors
    vectors= pd.DataFrame(temp_result2[:,1:], index=temp_result2[:,0],
    columns = [str(i) for i in range(1,len(temp_result2[0]),1)])
    # Sort the index of the created dataframe
    vectors_sorted = vectors.sort_index()
    return vectors_sorted

In [228]:
df = pgql_to_pandas(rs)

Converted to PGQL result set
Fetched all rows
Converted all rows to lists


In [229]:
df

,1,2,3,4,5,6
101,a01,Alice Nguyen,1,3,0.540319263320892,true
102,a02,Bob Tanaka,2,3,0.459680736679108,
103,a03,Carol Lee,1,2,0.0,
104,a04,,0,1,0.0,
105,a05,,0,0,0.0,
